# Post Training Quantization (PTQ) using AWQ

## This notebook demonstrates the process of Quantizing Large Language Models (LLMs) using the Activation-aware Weight Quantization (AWQ) technique.

## I saved the model weights and uploaded them to hugging face- 
https://huggingface.co/Mubinmodi007/Llama-3.2-1B-AWQ-4bit

## Evaluation and Explanation of the model is in LLM_Quant_testing_accuarcy.ipynb

Note: The execution of some cells might be not in order(had to run some cells because Colab session expired.



In [1]:
# Install necessary packages
!pip install autoawq
!pip install nvidia-ml-py3


INFO: pip is looking at multiple versions of autoawq-kernels to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 92.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 36.9

In [1]:
!pip uninstall -y torch torchvision torchaudio

!pip install torch==2.3.1 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

!pip install autoawq autoawq-kernels

Found existing installation: torch 2.3.1+cu118
Uninstalling torch-2.3.1+cu118:
  Successfully uninstalled torch-2.3.1+cu118
Found existing installation: torchvision 0.18.1+cu118
Uninstalling torchvision-0.18.1+cu118:
  Successfully uninstalled torchvision-0.18.1+cu118
Found existing installation: torchaudio 2.3.1+cu118
Uninstalling torchaudio-2.3.1+cu118:
  Successfully uninstalled torchaudio-2.3.1+cu118
Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torch-2.3.1%2Bcu118-cp310-cp310-linux_x86_64.whl (839.7 MB)
  Using cached https://download.pytorch.org/whl/cu118/torchvision-0.19.1%2Bcu118-cp310-cp310-linux_x86_64.whl (6.3 MB)
  Using cached https://download.pytorch.org/whl/cu118/torchaudio-2.4.1%2Bcu118-cp310-cp310-linux_x86_64.whl (3.3 MB)
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
  Using cached https://download.pyt

In [3]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer
from huggingface_hub import notebook_login, HfApi
from pynvml import nvmlInit, nvmlDeviceGetHandleByIndex, nvmlDeviceGetMemoryInfo

In [4]:
# Hugging Face API authentication
notebook_login()
username = "mubinmodi007"
api = HfApi(token="your_huggingface_token")  # Replace with your actual token

In [5]:
# Model and quantization parameters
model_path = "meta-llama/Llama-3.2-1B"
quant_path = "Llama-3.2-1B-AWQ-4bit"
quant_config = {"zero_point": True, "q_group_size": 128, "w_bit": 4}

In [8]:
# Load the model and tokenizer
model = AutoAWQForCausalLM.from_pretrained(model_path).cuda() # move model to cuda
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=True)

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

In [9]:


# Quantize the model
model.quantize(tokenizer, quant_config=quant_config)

Repo card metadata block was not found. Setting CardData to empty.
AWQ: 100%|██████████| 16/16 [05:53<00:00, 22.12s/it]


In [10]:
# Save the quantized model and tokenizer
model.save_quantized(f"./{quant_path}", safetensors=True)
tokenizer.save_pretrained(f"./{quant_path}")

Note that `shard_checkpoint` is deprecated and will be removed in v4.44. We recommend you using split_torch_state_dict_into_shards from huggingface_hub library


('./Llama-3.2-1B-AWQ-4bit/tokenizer_config.json',
 './Llama-3.2-1B-AWQ-4bit/special_tokens_map.json',
 './Llama-3.2-1B-AWQ-4bit/tokenizer.json')

In [17]:
# Hugging Face API authentication
notebook_login()
username = "Mubinmodi007"
api = HfApi(token="your_huggingface_token")  # Replace with your actual token

In [18]:
from huggingface_hub import HfApi

api = HfApi()
user_info = api.whoami()
print(f"Logged in as: {user_info['name']}")
print(f"Token permissions: {user_info['auth']['accessToken']['role']}")

Logged in as: Mubinmodi007
Token permissions: fineGrained


In [19]:
from huggingface_hub import create_repo

repo_url = create_repo(repo_id=f"{username}/{quant_path}", repo_type="model", private=False)
print(f"Repository created: {repo_url}")

Repository created: https://huggingface.co/Mubinmodi007/Llama-3.2-1B-AWQ-4bit


In [21]:
import os
# First, let's check the contents of the directory we're trying to upload
print(f"Contents of ./{quant_path}:")
print(os.listdir(f"./{quant_path}"))

# Now, let's try uploading the files
api.upload_folder(repo_id=f"{username}/{quant_path}", folder_path=f"./{quant_path}")
print("Files uploaded successfully")

Contents of ./Llama-3.2-1B-AWQ-4bit:
['special_tokens_map.json', 'config.json', 'generation_config.json', 'model.safetensors', 'tokenizer.json', 'tokenizer_config.json']


model.safetensors:   0%|          | 0.00/1.56G [00:00<?, ?B/s]

Files uploaded successfully


In [22]:
# Function to check GPU utilization
def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    return f"GPU memory occupied: {info.used // 1024**2} MB."

In [24]:
import torch
# Load the quantized model
model = AutoAWQForCausalLM.from_pretrained(f"{username}/{quant_path}", torch_dtype=torch.float16, device_map="auto")
print(print_gpu_utilization())

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/180 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.56G [00:00<?, ?B/s]

GPU memory occupied: 2943 MB.


In [25]:
# Check for bf16 support and set compute dtype
compute_dtype = torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16

In [26]:
# Load the original model for comparison (if needed)
model = AutoAWQForCausalLM.from_pretrained(model_path, torch_dtype=compute_dtype, device_map="auto")
print(print_gpu_utilization())

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

GPU memory occupied: 4063 MB.
